
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# Explore Scheduling Options

In the last lesson, we manually triggered our job. In this lesson, we will explore three other types of triggers we can use in our Databricks Workflow Jobs:
1. Scheduled
1. File arrival
1. Continuous

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:

1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

  - In the drop-down, select **More**.

  - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in a new tab*.

1. Find the triangle icon to the right of your compute cluster name and click it.

1. Wait a few minutes for the cluster to start.

1. Once the cluster is running, complete the steps above to select your cluster.

## A. Classroom Setup

Run the following cell to configure your working environment for this course. It will also set your default catalog to **dbacademy** and the schema to your specific schema name shown below using the `USE` statements.
<br></br>
```
USE CATALOG dbacademy;
USE SCHEMA dbacademy.<your unique schema name>;
```

**NOTE:** The **DA** object is only used in Databricks Academy courses and is not available outside of these courses.

In [0]:
%run ./Includes/Classroom-Setup-3

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


Course Catalog:,
Your Schema:,


**NOTE:** If you have already completed the demonstration and want to repeat it, uncomment and run the `DA.delete_baby_names_csv()` method to remove the CSV file used in this demonstration.

In [0]:
# DA.delete_baby_names_csv()

## B. Create a Job

The following method uses the Databricks SDK to programmatically create a job named **&lt;your-schema&gt;_Lesson03** for the demonstration. The task in the job uses the **Task Notebooks/Lesson 3 Notebooks/View Baby Names** notebook.

**NOTE:** You can find the method definition that uses the Databricks SDK to create the job in the [Classroom-Setup-3]($./Includes/Classroom-Setup-3) notebook. However, the [Databricks SDK](https://databricks-sdk-py.readthedocs.io/en/latest/) is outside the scope of this course.

1. Run the next cell to automatically set up the single-task job named **&lt;your-schema&gt;_Lesson03**. Confirm that the cell output created the job. If the job is already created, an error will be returned.


In [0]:
DA.create_job_lesson03()

Created the job: labuser9084188_1738337208_Lesson_03
Job ID: 675586138890978


2. View the new job (... **Lesson_03**):
   - a. Right-click on **Workflows** in the left navigation bar and select *Open Link in New Tab*.

   - b. Confirm that you see the job **&lt;your-schema&gt;_Lesson03**. Select the job to open it.

   - c. Select **Tasks** in the top navigation bar. The job should contain a single task named **View_New_CSV_Data**.

   - d. View the **Path** of the task. Confirm that the path is using the **Task Notebooks/Lesson 3 Notebooks/View Baby Names** 
   notebook.
   - e. View the **Compute** of the task. Confirm that it is using **Serverless**.

   - f. Leave the job page open and return to below instructions.


## C. Explore Scheduling Options
Steps:
1. Return to your job.

2. Make sure you are in the **Tasks** tab of your job.

3. On the right hand side of the Jobs UI, locate the **Job Details** section. Note if side panel is collapsed, click the left-hand arrowhead icon to expand it.

4. Under the **Schedules & Triggers** section, select the **Add trigger** button to explore the options. There are three options (in addition to manual):
   * **Scheduled** - uses a cron scheduling UI.
      - This UI provides extensive options for setting up chronological scheduling of your Jobs. Settings configured with the UI can also be output in cron syntax, which can be edited if you need custom configuration that is not available with the UI.

   * **Continuous** - runs over and over with a small amount of time between runs.

   * **File arrival** - monitors either an external location or a volume for new files. Note the **Advanced** settings, where you can change the time to wait between checks and the time to wait after a new file arrives before starting a run.

5. Leave the **Schedules & Triggers** open and return to below instructions.

## D. Configure the File Arrival Trigger

Let's configure a file arrival trigger to monitor a volume for new data files.

1. Start by running the cell below to create a volume named **trigger_storage_location**, which we will use as the storage location to monitor. The volume will be created in the **dbacademy** catalog within your unique schema.

**NOTE:**  Databricks volumes are Unity Catalog objects representing a logical volume of storage in a cloud object storage location. Volumes provide capabilities for accessing, storing, governing, and organizing files. You can use volumes to store and access files in any format, including structured, semi-structured, and unstructured data.


In [0]:
%sql
CREATE VOLUME IF NOT EXISTS trigger_storage_location


2. View your new volume:
- a. Select the catalog icon on the left and navigate to your schema in the **dbacademy** catalog. Expand your schema.

- b. In your schema expand **Volumes**. Confirm that the **trigger_storage_location** volume was created.

- c. Expand the **trigger_storage_location** volume. Confirm that the volume does not contain any files.

You can also use the `SHOW VOLUMES` statement to view available volumes in your schema.

In [0]:
%sql
SHOW VOLUMES;

database,volume_name
labuser9084188_1738337208,trigger_storage_location


3. Run the following cell to get the path to this volume using the custom `DA` object created for this course.

    **NOTE:** You can also select your volume under the catalog, click the three ellipses, and then select *Copy volume path* to get the volume path.

In [0]:
your_volume_path = (f"/Volumes/{DA.catalog_name}/{DA.schema_name}/trigger_storage_location/")
print(your_volume_path)

/Volumes/dbacademy/labuser9084188_1738337208/trigger_storage_location/


4. Complete the following to configure the **File Arrival** trigger on your job:

- a. Navigate back to the browser tab with your job.

- b. In your job, under **Trigger type**, select **File Arrival** for the trigger type.

- c. Paste the path above into the **Storage location** field

- d. Click **Test Trigger** to verify the correct path

  **NOTE:** You should see **Success**. If not, verify that you have run the cell above and copied all of the cell output into **Storage location**

- e. Expand the **Advanced** options. Notice that you can set different trigger options.

- f. Click **Save**

## E. Set Task Parameters

The notebook we will use to view our baby names needs to know the name of the catalog and schema we are working with. We can configure this using **Task parameters**. This provides flexibility and the ability to reuse code.

1. Run the cell below to view your **catalog** and **schema** names.

In [0]:
print(f"Catalog name: {DA.catalog_name}")
print(f"Schema name: {DA.schema_name}")

Catalog name: dbacademy
Schema name: labuser9084188_1738337208


2. Set the task parameters:

- a. Go back to your job. In **Task details** pane, under **Parameters**, click **Add**.

- b. For **Key**, type *catalog*, and for the **Value** type *dbacademy*. 

- c. Repeat the steps above for the schema name, using the **Key** *schema* and your schema name from the above cell output for the **Value**.

- d. Click **Save task**.

- e. Click to open the [Task Notebooks/Lesson 3 Notebooks/View Baby Names]($./Task Notebooks/Lesson 3 Notebooks/View Baby Names) notebook. This notebook is used in the **View_New_CSV_Data** task. Notice the following:
   - The **my_catalog** variable is obtaining the value from the **catalog** parameter we set in the task using the following:

      -  Creates a text input widget in the notebook using the **catalog** parameter:
         - `dbutils.widgets.text(name='catalog', defaultValue='')`
      - Stores the values from the text input into the variable **my_catalog**:
         - `my_catalog = dbutils.widgets.get('catalog')`

   - The **my_schema** variable is obtaining the value from the **schema** parameter we set in the task using the following: 
      -  Creates a text input widget in the notebook using the **schema** parameter:
         - `dbutils.widgets.text(name='schema', defaultValue='')`
      - Stores the values from the text input into the variable **my_schema**:
         - `my_schema = dbutils.widgets.get('schema')`

   - The **my_volume_path** variable uses the parameters we set to point to your **trigger_storage_location** volume using:
      - `f"/Volumes/{my_catalog}/{my_schema}/trigger_storage_location/"`

- f. Close the **View Baby Names** notebook


**Example**

![Lesson03_JobTrigger](files/images/deploy-workloads-with-databricks-workflows-2.0.1/Lesson03_JobTrigger.png)

3. As soon as we configured our trigger, Databricks began monitoring the storage location for newly arrived files (by default, it will check every one minute). Let's take a look at the status of our job runs.
  - a. In the upper-left corner, click the **Runs** tab
We should see a **Trigger status**. If not, wait about a minute. If you don't see one during that time, double-check the steps above to ensure you configured the **File arrival** trigger correctly.

  - b. Note that the trigger has been evaluated, but it has not found any new files, so the job has not run.

  - c. Run the cell below to add a CSV file to your **trigger_storage_location** volume, and wait about 1-2 minutes. You should see a run triggered automatically.

  - d. After the job completes (about 1-2 minutes), click on the **Start time** to view the run. The notebook simply displays the contents of the CSV file.


  **NOTE:** You can manually trigger a run using different parameters by going to the job configuration page (click **Edit task** from the **Run output** page), clicking the down arrow next to **Run now** and selecting **Run now with different parameters**.

In [0]:
import requests

## Sends an HTTP GET request to the provided URL and stores the response
response = requests.get("https://health.data.ny.gov/api/views/jxy9-yhdk/rows.csv")

## Converts the byte data into a string using UTF-8 encoding in the variable csvfile
csvfile = response.content.decode("utf-8")

## Uploads the CSV data stored in the csvfile variable to a a volume in your schema
dbutils.fs.put(f"{your_volume_path}/babynames.csv", csvfile, True)

Wrote 2258987 bytes.


True

## F. Delete the Job

1. Navigate back to all of your jobs.

2. Find the job you just created, **&lt;Your-Schema &gt;_Lesson_03**.

3. To the right of the job, select the three ellipses and choose **Delete job**.


&copy; 2025 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>